In [1]:
import numpy as np
import tensorflow as tf
import os
import pandas as pd

c:\program files (x86)\python\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\program files (x86)\python\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\program files (x86)\python\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\program files (x86)\python\lib\site-packages\tensorflow\python\framework\dtypes.py:519: 

In [2]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(16,input_dim=13, kernel_initializer='normal', activation='relu'),
  tf.keras.layers.Dense(32,kernel_initializer='normal', activation='relu'),
  tf.keras.layers.Dense(32,kernel_initializer='normal', activation='relu'),
  tf.keras.layers.Dense(8,kernel_initializer='normal', activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])
adam = tf.keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=adam,
                loss='binary_crossentropy',
                metrics=['acc'])
print(model.summary())

W0518 11:59:28.663754 12092 deprecation.py:506] From c:\program files (x86)\python\lib\site-packages\tensorflow\python\keras\initializers.py:143: calling RandomNormal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0518 11:59:28.666771 12092 deprecation.py:506] From c:\program files (x86)\python\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0518 11:59:28.791681 12092 deprecation.py:323] From c:\program files (x86)\python\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.a

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                224       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 264       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 9         
Total params: 2,097
Trainable params: 2,097
Non-trainable params: 0
_________________________________________________________________
None


In [3]:
checkpoint_filepath = './best_model/ckpt{epoch}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_filepath)
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)
model.load_weights(latest)

./best_model\ckpt_52


In [4]:
# cek lagi (don't put this block in build app)
dataset = pd.read_csv('heart.csv', sep=';')
data = dataset[~dataset.isin(['?'])]
data = data.dropna(axis=0)
# Ubah target 2,3,4 jadi 1
data.loc[data['target'] > 1, 'target'] = 1
data = data.apply(pd.to_numeric)
X = np.array(data.drop(['target'], 1))
y = np.array(data['target'])
# Re-evaluate the model
loss,acc = model.evaluate(X,  y, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

data.head()

297/297 - 0s - loss: 0.3746 - acc: 0.8586
Restored model, accuracy: 85.86%


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,1
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


In [5]:
predArr = np.array([[ 67, 1, 4, 160, 286, 0, 2, 108, 1, 1.5, 2, 3, 3 ]])
print(predArr.shape)
predictions = model.predict(predArr)

(1, 13)


In [6]:
print(predictions[0])
if predictions[0]<0.5:
    print("0")
elif predictions[0]>=0.5:
    print("1")

[0.9755711]
1
